In [ ]:
import time
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import models, transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torch.utils.data import DataLoader
# Dataset
from CustomDataset import CatDataset

In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print('Is CUDA installed ?:',torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('CUDA Version:',torch.version.cuda)
    print('Using GPU.')
else:
  print('No CUDA is installed, using CPU instead.') 
  device = torch.device('CPU')

-----------------------------------
**No need to use anything below yet.**
**Reserved for future run.**
-----------------------------------



In [ ]:
#Generate label file
DATA_PATH = 'catset/labels'
labelset = []
for label in os.listdir(DATA_PATH):
    end = len(label)
    label = label[:(end-4)]+'.jpg'
    labelset.append(label)
    print(label)

In [ ]:
df = pd.DataFrame(labelset)
df.insert(1, 'label', '0')
df.reset_index(drop=True)
df.head(5)
df.to_csv('catset/labels.csv',
            index=False,
            header=False    )

In [23]:
import secrets
import shutil
IMG_DIR = 'catset/images/'
LABEL_DIR = 'catset/labels/'
VAL_DIR = 'catset/val/'
VAL_LABEL_DIR = 'catset/valbal/'
val_percent = 16

for img in os.listdir(IMG_DIR):
    if secrets.randbelow(101)<val_percent:
        shutil.move(IMG_DIR+img,VAL_DIR+img)
        if os.path.isfile(LABEL_DIR+img.split('.')[0]+'.txt'):
            shutil.move(LABEL_DIR+img.split('.')[0]+'.txt',VAL_LABEL_DIR+img.split('.')[0]+'.txt')

In [40]:
for label in os.listdir(VAL_LABEL_DIR):
    if not os.path.isfile(VAL_DIR+label[:len(label)-4]+'.jpg'):
        print('error')

In [ ]:
training_data = CatDataset(csv_file='catset/labels.csv', root_dir='catset/images',transform=ToTensor)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [ ]:
catto = torch.hub.load('ultralytics/yolov5', 'custom', path='weight/YoloV5s_cat.pt',force_reload=True)  # Load Catto model

In [ ]:
IMG_PATH = 'training_data/images/'
imgset = []
for img in os.listdir(IMG_PATH):
    imgset.append(IMG_PATH+img)
len(imgset)

In [ ]:
catto.cpu()
results = catto(imgset, size=320)  # custom inference size
results.xyxy[0]


In [ ]:
results.save()


Training line:

```

python train.py --img 320  --batch 32  --epochs 100  --data catset.yaml --worker 0  --weights yolov5s.pt

```
```
python train.py --img 320  --batch 32  --epochs 100  --data catset.yaml --worker 0  --weights weights/catto.pt
```
Freezing line (replace in train.py):

```

freeze =  ['model.%s.'  % x for x in  range(10)]

```

Test line:

```

#Using Yolov5s model to test

python detect.py --weights yolov5s.pt --img 640  --conf 0.25  --source catset/images

#Using Catto model to test

python detect.py --weights yolov5s_cat.pt --img 320  --source catset/images

```